In [1]:
# coding: utf-8
import re 
import pandas as pd
import requests
import random
import json
from urllib import parse
from urllib import request
import time
from datetime import datetime
from datetime import timedelta
import os

In [4]:
header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Connection':'keep-alive'
}

### 获取种类

In [54]:
def get_kinds():

    url = 'https://www.gushiwen.org/shiwen'
    html = requests.get(url=url, headers=header).content.decode('utf-8')
    #re.S  它表示“.”（不包含外侧双引号，下同）的作用扩展到整个字符串，包括“\n”
    s = r'<div class=\"sright\">\n(.*?)\n<\/div>'
    pages = re.findall(s, html,re.S)
    raw_hrefs = pages[0].split("\n")
    content=[]
    for raw_href in raw_hrefs:
        s2= r'<a .*?href=\"(.*?)\">(.*?)</a>'
        content=content+re.findall(s2, raw_href)
    return content


In [55]:
get_kinds()

[('/shiwen/default_1Ab26259653a5dA1.aspx', '写景'),
 ('/shiwen/default_1Ac606a404de17A1.aspx', '咏物'),
 ('/shiwen/default_1A5cc29cb69571A1.aspx', '春天'),
 ('/shiwen/default_1Abb09e8221916A1.aspx', '夏天'),
 ('/shiwen/default_1A815cd38f55bcA1.aspx', '秋天'),
 ('/shiwen/default_1A5d020748ba38A1.aspx', '冬天'),
 ('/shiwen/default_1Aa4ab34426306A1.aspx', '写雨'),
 ('/shiwen/default_1A9f8d7ab22bbcA1.aspx', '写雪'),
 ('/shiwen/default_1Ab60ee740c2f4A1.aspx', '写风'),
 ('/shiwen/default_1A226640f821dcA1.aspx', '写花'),
 ('/shiwen/default_1Afba9d4850b2eA1.aspx', '梅花'),
 ('/shiwen/default_1A54b02c5dad26A1.aspx', '荷花'),
 ('/shiwen/default_1A613485d2b7a1A1.aspx', '菊花'),
 ('/shiwen/default_1A4b33c9385f57A1.aspx', '柳树'),
 ('/shiwen/default_1A8f86db5a929cA1.aspx', '月亮'),
 ('/shiwen/default_1Ae2fd13ba92c6A1.aspx', '山水'),
 ('/shiwen/default_1Af8646c491975A1.aspx', '写山'),
 ('/shiwen/default_1Af32f819cb6afA1.aspx', '写水'),
 ('/shiwen/default_1A294d794fa505A1.aspx', '长江'),
 ('/shiwen/default_1A4f3c7ecbd61eA1.aspx', '黄河'),


### 根据种类获取页数

In [58]:
def get_pages_by_kind(kind):
    try:
        url = 'https://www.gushiwen.org'+kind[0]
        html = requests.get(url=url, headers=header).content.decode('utf-8')
        s = r'sumPage\" style=\"display:none;\">(.*?)<\/label>'
        pages = re.findall(s, html)
        return pages[0]
    except Exception as e:
        return 0

In [59]:
get_pages_by_kind(('/shiwen/default_1A589282347eb3A1.aspx', '边塞'))

'18'

In [70]:
def get_poem_by_kind(kind):
    
    poetry = pd.DataFrame(columns=['author','dynasty','title','content'])
    pages = get_pages_by_kind(kind)
    path= r'C:\\Users\\HP\\新建文件夹\\pa_chong\\poem\\类型\\'+kind[1]+'.csv'
    
    if int(pages)>0:
#       不存在文件夹自动生成
        if not os.path.exists('类型'):
            os.mkdir('类型')
        poetry.to_csv(path,index=False,encoding="utf_8_sig")
        print("正在爬取种类    "+kind[1])
    else:
        
        print(kind[1]+'    未找到种类')
    for i in range(0,int(pages)):
            time.sleep(0.2)
            url='https://www.gushiwen.org'+kind[0]

            html = requests.get(url=url,headers=header).content.decode('utf-8')
            s1= r'<textarea(.*?)<\/textarea>'
            content = re.findall(s1, html)
            print(str(i)+" "+url+"\t"+str(len(content)))
            
            for j in range(0,len(content)):
                try:
                    item0=content[j].split('>')[1]
                    item1=item0.split('——')
                    # 获取内容
                    poem_content=item1[0]
                    #获取题目
                    poem_title=item1[1].split('《')[1].split('》')[0]
                    # 获取作者
                    poem_author = item1[1].split('《')[0].split('·')[1]
                    # 获取作者朝代
                    poem_author_dynasty = item1[1].split('《')[0].split('·')[0]

                    poetry = poetry.append({'author':poem_author,'dynasty':poem_author_dynasty,
                                                'title':poem_title,'content':poem_content},
                                                ignore_index=True)
                except Exception as e:
                    print("1Erroe:")
                    print(e)
                    continue
            poetry.to_csv(path,index=False,mode='a', header=False,encoding="utf_8_sig")
            poetry.drop(poetry.index,inplace=True)#清空c
            
#     except Exception as e:
#         print("2Erroe:")
#         print(e)
#         time.sleep(0.5)

In [71]:
get_poem_by_kind(('/shiwen/default_1A589282347eb3A1.aspx', '边塞'))

正在爬取种类    边塞
0 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
5 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
9 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
10 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1A589282347eb3A1.aspx	10
14 https://www.gushiwen.org/shiwen/default_1A

In [73]:
kinds = get_kinds()
for kind in kinds:
    get_poem_by_kind(kind)

正在爬取种类    写景
0 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
5 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
9 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
10 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
14 https://www.gushiwen.org/shiwen/default_1A

121 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
122 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
123 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
124 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
125 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
126 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
127 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
128 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
129 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
130 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
131 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
132 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
133 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
134 https://www.gushiwen.org/shiwen/default_1Ab26259653a5dA1.aspx	10
135 https://www.gushiwen.org/shiwe

17 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
18 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
19 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
20 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
21 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
22 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
23 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
24 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
25 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
26 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
27 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
28 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
29 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
30 https://www.gushiwen.org/shiwen/default_1A5cc29cb69571A1.aspx	10
31 https://www.gushiwen.org/shiwen/default_1A5cc

10 https://www.gushiwen.org/shiwen/default_1Afba9d4850b2eA1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1Afba9d4850b2eA1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1Afba9d4850b2eA1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1Afba9d4850b2eA1.aspx	10
正在爬取种类    荷花
0 https://www.gushiwen.org/shiwen/default_1A54b02c5dad26A1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1A54b02c5dad26A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1A54b02c5dad26A1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1A54b02c5dad26A1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1A54b02c5dad26A1.aspx	10
正在爬取种类    菊花
0 https://www.gushiwen.org/shiwen/default_1A613485d2b7a1A1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1A613485d2b7a1A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1A613485d2b7a1A1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1A613485d2b7a1A1.aspx	10
正在爬取种类    柳树
0 https://www.gushiwen.org/shiwen/default_1A4b33c9385f57A1.aspx	10
1 https://www.gushi

5 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
9 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
10 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
14 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
15 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
16 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
17 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
18 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
19 https://www.gushiwen.org/shiwen/default_1A7b021696

126 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
127 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
128 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
129 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
130 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
131 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
132 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
133 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
134 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
135 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
136 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
137 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
138 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
139 https://www.gushiwen.org/shiwen/default_1A7b021696e0f4A1.aspx	10
140 https://www.gushiwen.org/shiwe

10 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
14 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
15 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
16 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
17 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
18 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
19 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
20 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
21 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
22 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
23 https://www.gushiwen.org/shiwen/default_1Ab32e58b2d593A1.aspx	10
24 https://www.gushiwen.org/shiwen/default_1Ab32

19 https://www.gushiwen.org/shiwen/default_1A3f1bda9479bcA1.aspx	10
正在爬取种类    闺怨
0 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
5 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
9 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
10 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1A4c55e41cf4beA1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1A

1 https://www.gushiwen.org/shiwen/default_1A589930174417A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1A589930174417A1.aspx	10
正在爬取种类    惜时
0 https://www.gushiwen.org/shiwen/default_1Aaec7d44486b8A1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1Aaec7d44486b8A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1Aaec7d44486b8A1.aspx	10
正在爬取种类    婉约
0 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
5 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1Ad39bd9e8fff1A1.aspx	10
9 https://www.gushiwen.org/shiwen/de

7 https://www.gushiwen.org/shiwen/default_1Afb7ed84b746fA1.aspx	10
正在爬取种类    忧国忧民
0 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
1 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
2 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
3 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
4 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
5 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
6 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
7 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
8 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
9 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
10 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
11 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
12 https://www.gushiwen.org/shiwen/default_1Ac2c5e4ae196aA1.aspx	10
13 https://www.gushiwen.org/shiwen/default_1